In [1]:
# !pip install transformers
# !pip install datasets
import os
import re
import soundfile as sf
import torch
import numpy as np
from datasets import load_dataset, load_metric, Dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import pandas as pd
import string
import subprocess

In [2]:
# load pretrained model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h").to("cuda")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
def train_test_split(ds, test_size = 0.3):
    n = len(ds)
    idx = np.random.permutation(n)
    train = ds.select(idx[round(n*test_size):])
    test = ds.select(idx[:round(n*test_size)])
    return train, test

In [4]:
DATA_PATH = "K:\\AIPI540\\Individual Project\\cv-corpus-8.0-2022-01-19-en\\cv-corpus-8.0-2022-01-19\\en\\clips"
LABEL_PATH = "K:\\AIPI540\\Individual Project\\cv-corpus-8.0-2022-01-19-en\\cv-corpus-8.0-2022-01-19\\en"
CLEAN_AUDIO = "E:\\Graduate\\2021-2022 Term 2\\AIPI540\\Individual Project\\Data\\Train"
SIZE = 10000
SAMPLE_RATE = 16000

In [5]:
labels = pd.read_csv(os.path.join(LABEL_PATH, "train.tsv"), sep = "\t").loc[:SIZE, ['path', 'sentence']]

In [6]:
def label_preprocess(line):
    line['path'] = os.path.join(CLEAN_AUDIO, line['path'].split(".")[0]+"-normalized_Vocals.wav")
    line['sentence'] = re.sub('[^a-zA-Z]+',"",line['sentence']).upper()
    return line

In [7]:
processed_labels = labels.apply(label_preprocess, axis = 1)
processed_labels.head()

,path,sentence
0,E:\Graduate\2021-2022 Term 2\AIPI540\Individua...,THEREAFTERTHECLASSWASHIGHLYRESPECTED
1,E:\Graduate\2021-2022 Term 2\AIPI540\Individua...,BANARASHINDUUNIVERSITYISACENTRALUNIVERSITYINPU...
2,E:\Graduate\2021-2022 Term 2\AIPI540\Individua...,ONDISPLAYAREHOMEFURNISHINGSPIONEERTOOLSANDRANC...
3,E:\Graduate\2021-2022 Term 2\AIPI540\Individua...,ELEVAANDSTRUMEACHHOUSEANELEMENTARYSCHOOL
4,E:\Graduate\2021-2022 Term 2\AIPI540\Individua...,THEEASTERNPORTIONOFTHECOUNTYLIESWITHINTHEBLUER...


In [8]:
processed_labels['path'][0]

'E:\\Graduate\\2021-2022 Term 2\\AIPI540\\Individual Project\\Data\\Train\\common_voice_en_28449980-normalized_Vocals.wav'

In [9]:
dataset = Dataset.from_pandas(processed_labels[:100])

In [10]:
def prepare_dataset(batch):
    # audio = batch["audio"]
    audio_input, sample_rate = sf.read(batch["path"])
    # batched output is "un-batched"
    batch["input_values"] = processor(audio_input, sampling_rate=sample_rate).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [11]:
model.freeze_feature_encoder()

In [12]:
train, test = train_test_split(dataset)
train_pro = train.map(prepare_dataset, remove_columns=train.column_names)
test_pro = test.map(prepare_dataset, remove_columns=test.column_names)

  6%|▌         | 4/70 [00:03<00:59,  1.11ex/s]


RuntimeError: Error opening 'E:\\Graduate\\2021-2022 Term 2\\AIPI540\\Individual Project\\Data\\Train\\common_voice_en_19477771-normalized_Vocals.wav': System error.